In [1]:
%%timeit -r 10 -n 30

import psycopg2

# Parametri di connessione al database PostgreSQL
db_params = {
    'host': 'localhost',
    'database': 'database4test',
    'user': 'postgres',
    'password': 'oirad1993'
}

# Connessione al database
conn = psycopg2.connect(**db_params)
cursor = conn.cursor()

# Query SQL
sql_query = """
    SELECT ae_inner."nomemodello"
    FROM "apparecchiatureelettroniche" ae_inner
    WHERE (
        SELECT digest(CONCAT(ae_inner."manualeutente", ae_inner."nomemodello"), 'sha1')
        ) = (
        SELECT digest(CONCAT(ae."manualeutente", ae."nomemodello"), 'sha1')
        FROM "apparecchiatureelettroniche" ae
        WHERE ae."id" = ae_inner."id"
        )
        AND LENGTH(ae_inner."manualeutente") > 0
        AND ae_inner."id" IN (
            SELECT po."id"
            FROM "prodottiordinati" po
            WHERE po."numeroordine" IN (
                SELECT oc."numeroordine"
                FROM "ordiniclienti" oc
                WHERE oc."cittaspedizione" = 'Milano'
                    AND oc."dataordine" BETWEEN '2023-01-01' AND '2023-07-31'
            )
        );
"""

# Esecuzione della query
cursor.execute(sql_query)

# Recupero dei risultati
results = cursor.fetchall()

# Chiusura della connessione
cursor.close()
conn.close()

# Stampa dei risultati
for result in results:
    print(result[0])


103 ms ± 4.96 ms per loop (mean ± std. dev. of 10 runs, 30 loops each)
